In [9]:
import requests
import pandas as pd
import functions as f
from barentswatch.credentials import config
from barentswatch.authentication import get_token

In [10]:
import functions as f
import warnings
from pprintpp import pprint
warnings.simplefilter(action='ignore', category=FutureWarning)

In [17]:
def get_week_summary(token, year, week):
  url = f"{config['api_base_url']}/v1/geodata/fishhealth/locality/{locality}/{year}/{week}"
  headers ={
    'authorization': 'Bearer ' + token['access_token'],
    'content-type': 'application/json',
  }

  response = requests.get(url, headers=headers)
  response.raise_for_status()
  return response.json()

In [18]:
token = get_token()
weeksummary= get_week_summary(token,'2017','45')
pprint(weeksummary)

Token request successful
{
    'aquaCultureRegister': {
        'aquaCultureUpdateTime': '2017-11-06T00:00:00+01:00',
        'capacity': 10.0,
        'isGreen': False,
        'lat': 65.045867,
        'licenses': [
            {
                'capacity': 10.0,
                'expirationDate': None,
                'grantDate': '2002-02-28T00:00:00',
                'isGreen': False,
                'licenseNo': 'N BL0321',
                'licensee': 'AARSAND, ARILD',
                'localities': [
                    {
                        'isReportingLocality': True,
                        'localityNo': 14746,
                        'municipality': 'Bindal',
                        'name': 'Aarsand',
                    },
                ],
                'productionType': 'Akvakulturdyr til konsum',
                'purpose': 'Kommersiell',
                'species': ['Blåskjell'],
                'unit': 'DA',
            },
        ],
        'localityNo': 14746,
   